# Introduction: Internal Rate of Return (IRR) and Solution Methods

The **Internal Rate of Return (IRR)** is a fundamental financial metric used to evaluate the profitability of investments or projects. IRR is defined as the discount rate that makes the Net Present Value (NPV) of a series of cash flows equal to zero:

$$
NPV = \sum_{t=0}^{N} \frac{C_t}{(1 + IRR)^t} = 0
$$

where \( C_t \) is the cash flow at time \( t \).

## Methods for Solving IRR

There is no general analytical solution for IRR with arbitrary cash flows, so it must be found using numerical methods. The most common approaches include:

- **Newton-Raphson Method:** An iterative root-finding algorithm that converges quickly if the initial guess is close to the true IRR. Used by libraries such as [numpy-financial's irr](https://numpy.org/numpy-financial/latest/irr.html) and many financial calculators.
- **Bisection Method:** A slower but more robust iterative method that always converges if the function changes sign over the interval.
- **Centroid-based Newton-Raphson:** An improved variant of Newton-Raphson that uses the centroid (average) of the current and next guesses to potentially reduce the number of iterations required for convergence.

Different software tools may use different algorithms and stopping criteria. For example, Microsoft Excel’s IRR function uses an iterative approach and **has important limitations**:

- **Iteration Cap:** Excel’s IRR algorithm stops after 20 iterations if it does not converge, returning a `#NUM!` error (effectively a NaN).
- **Sensitivity to Initial Guess:** If the initial guess is not close to the actual IRR, Excel may fail to find a solution within the iteration limit.
- **Multiple Roots:** If there are multiple IRRs (possible with non-standard cash flows), Excel may return only one or fail to converge.

In contrast, some libraries (such as [node-irr](https://github.com/eric-malachias/irr)) use Newton-Raphson with a fallback to bisection and do not cap the number of iterations as strictly, making them more robust for complex cash flow scenarios.

This notebook demonstrates IRR calculation and compares these numerical methods, highlighting the differences and limitations of each approach.

In [ ]:
### IRR calculation using centroid-based Newton-Raphson method, lets test it
import numpy as np


In [ ]:
# Define the function f(r) for NPV, where r is the IRR
def npv(r, cash_flows):
    return np.sum(cash_flows / (1 + r) ** np.arange(len(cash_flows)))


# Define the derivative of the function f'(r) for NPV
def npv_derivative(r, cash_flows):
    t = np.arange(1, len(cash_flows))  # Exclude the initial time point (t=0)
    return -np.sum(t * cash_flows[1:] / (1 + r) ** (t + 1))


# Centroid-based Newton-Raphson method for IRR
def centroid_newton_raphson_irr(cash_flows, initial_guess, tol=1e-6, max_iter=100):
    r_curr = initial_guess  # Initial guess for IRR

    for i in range(max_iter):
        # Evaluate the function and its derivative at the current guess
        f_val = npv(r_curr, cash_flows)
        df_val = npv_derivative(r_curr, cash_flows)

        # Calculate the next estimate using Newton-Raphson
        r_next = r_curr - f_val / df_val

        # Calculate the centroid-based update
        centroid = np.mean([r_curr, r_next])  # Centroid of current and next guesses

        # Check for convergence
        if np.abs(npv(centroid, cash_flows)) < tol:
            print(f"Converged to IRR: {centroid:.6f} in {i + 1} iterations.")
            return centroid

        # Update the guess with centroid
        r_curr = centroid

    print(f"Max iterations reached. Last estimate: {r_curr}")
    return r_curr


def irr_np_finance(values):
    """
    Return the Internal Rate of Return (IRR).

    This is the "average" periodically compounded rate of return
    that gives a net present value of 0.0; for a more complete explanation,
    see Notes below.

    :class:`decimal.Decimal` type is not supported.

    Parameters
    ----------
    values : array_like, shape(N,)
        Input cash flows per time period.  By convention, net "deposits"
        are negative and net "withdrawals" are positive.  Thus, for
        example, at least the first element of `values`, which represents
        the initial investment, will typically be negative.

    Returns
    -------
    out : float
        Internal Rate of Return for periodic input values.

    Notes
    -----
    The IRR is perhaps best understood through an example (illustrated
    using np.irr in the Examples section below).  Suppose one invests 100
    units and then makes the following withdrawals at regular (fixed)
    intervals: 39, 59, 55, 20.  Assuming the ending value is 0, one's 100
    unit investment yields 173 units; however, due to the combination of
    compounding and the periodic withdrawals, the "average" rate of return
    is neither simply 0.73/4 nor (1.73)^0.25-1.  Rather, it is the solution
    (for :math:`r`) of the equation:

    .. math:: -100 + \\frac{39}{1+r} + \\frac{59}{(1+r)^2}
     + \\frac{55}{(1+r)^3} + \\frac{20}{(1+r)^4} = 0

    In general, for `values` :math:`= [v_0, v_1, ... v_M]`,
    irr is the solution of the equation: [G]_

    .. math:: \\sum_{t=0}^M{\\frac{v_t}{(1+irr)^{t}}} = 0

    References
    ----------
    .. [G] L. J. Gitman, "Principles of Managerial Finance, Brief," 3rd ed.,
       Addison-Wesley, 2003, pg. 348.

    """
    # `np.roots` call is why this function does not support Decimal type.
    #
    # Ultimately Decimal support needs to be added to np.roots, which has
    # greater implications on the entire linear algebra module and how it does
    # eigenvalue computations.
    res = np.roots(values[::-1])

    mask = (res.imag == 0) & (res.real > 0)
    if not mask.any():
        return np.nan
    res = res[mask].real

    print(f"Roots found: {res}")
    # NPV(rate) = 0 can have more than one solution so we return
    # only the solution closest to zero.
    rate = 1 / res - 1
    rate = rate.item(np.argmin(np.abs(rate)))
    return rate


In [ ]:
initial_guess = 0.1  # Initial guess for IRR (typically a small positive value like 0.1 or 0.05) 10%
# Define the cash flows (example)
cash_flows = np.array(
    [
        -1000,
        200,
        300,
        400,
        500,
        600,
        700,
        800,
        900,
        1000,
        1100,
        1200,
        1300,
        1400,
        1500,
        1600,
        1700,
        1800,
        1900,
        2000,
    ]
)  # Example: initial investment and subsequent cash flows


In [ ]:
base_irr = irr_np_finance(cash_flows)
print(f"Calculated IRR (finance): {base_irr:.6f}")


In [ ]:
# Calculate IRR using centroid-based Newton-Raphson method
irr_centroid = centroid_newton_raphson_irr(cash_flows, initial_guess)
print(f"Calculated IRR (centroid): {irr_centroid:.6f}")


### typescript example

```typescript
// Define the cash flows (example)
const cashFlows: number[] = [-1000, 200, 300, 400, 500];  // Example: initial investment and subsequent cash flows

// Define the function f(r) for NPV, where r is the IRR
function npv(r: number, cashFlows: number[]): number {
    let result = 0;
    for (let t = 0; t < cashFlows.length; t++) {
        result += cashFlows[t] / Math.pow(1 + r, t);
    }
    return result;
}

// Define the derivative of the function f'(r) for NPV
function npvDerivative(r: number, cashFlows: number[]): number {
    let result = 0;
    for (let t = 1; t < cashFlows.length; t++) { // t starts at 1 to exclude initial investment
        result -= t * cashFlows[t] / Math.pow(1 + r, t + 1);
    }
    return result;
}

// Centroid-based Newton-Raphson method for IRR
function centroidNewtonRaphsonIRR(cashFlows: number[], initialGuess: number, tol: number = 1e-6, maxIter: number = 100): number {
    let rCurr: number = initialGuess; // Initial guess for IRR
    
    for (let i = 0; i < maxIter; i++) {
        // Evaluate the function and its derivative at the current guess
        const fVal = npv(rCurr, cashFlows);
        const dfVal = npvDerivative(rCurr, cashFlows);
        
        // Calculate the next estimate using Newton-Raphson
        let rNext = rCurr - fVal / dfVal;
        
        // Calculate the centroid-based update
        const centroid = (rCurr + rNext) / 2;  // Centroid of current and next guesses
        
        // Check for convergence
        if (Math.abs(npv(centroid, cashFlows)) < tol) {
            console.log(`Converged to IRR: ${centroid.toFixed(6)} in ${i + 1} iterations.`);
            return centroid;
        }
        
        // Update the guess with centroid
        rCurr = centroid;
    }

    console.log(`Max iterations reached. Last estimate: ${rCurr.toFixed(6)}`);
    return rCurr;
}

// Initial guess for IRR (typically a small positive value like 0.1 or 0.05)
const initialGuess = 0.1;  // 10%

// Calculate IRR using centroid-based Newton-Raphson method
const irr = centroidNewtonRaphsonIRR(cashFlows, initialGuess);
console.log(`Calculated IRR: ${irr.toFixed(6)}`);
```